## Описание проекта

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Обзор данных

In [3]:
# Анализ данных
import pandas as pd
import numpy as np
from IPython.display import display


**Описание данных df**
* `childre` — количество детей в семье.
* `days_employed` — общий трудовой стаж в днях.
* `dob_years` — возраст клиента в годах.
* `education` — уровень образования клиента.
* `education_id` — идентификатор уровня образования.
* `family_status` — семейное положение.
* `family_status_id` — идентификатор семейного положения.
* `gender` — пол клиента.
* `income_type` — тип занятости.
* `debt` — имел ли задолженность по возврату кредитов.
* `total_income` — ежемесячный доход.
* `purpose` — цель получения кредита.

In [4]:
display(df.head())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [5]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В результате обзора данных выявлены аномальные значения столбца `days_employed` - общий трудовой стаж не может иметь отрицательное количество дней.

Данные столбцов `days_employed` и `total_income` утрачены.

## Предобработка данных

### Предобработка данных. Заполнение пропущенных значений

In [6]:
# Поиск пропущенных значений
df.isna().sum()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Входе предобработки данных обнаружено, что в столбцах количества отработанных дней и общая прибыль пропущены значения. Оба столбца имеют одинаковое количество пропусков.

In [7]:
# Доля пропущенных значений 
print('Доля пропущенных значений days_employed:',df['days_employed'].isna().sum()/df.shape[0])
print('Доля пропущенных значений total_income:',df['total_income'].isna().sum()/df.shape[0])


Доля пропущенных значений days_employed: 0.10099883855981417
Доля пропущенных значений total_income: 0.10099883855981417


Доля пропущенных значений в каждом из двух столбцов равна 10%.

In [8]:
df.isna().mean()


children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

Вероятно автоматизированная система сбора информации дала сбой или клиент сам забыл их указать, что послужило причиной пропущенных значений.

In [9]:
# Заполнение пропущенных значений days_employed и total_income медианным значением
days_employed_median = df.days_employed.sort_values().median()
df.days_employed = df.days_employed.fillna(days_employed_median)

total_income_median = df.total_income.sort_values().median()
df.total_income = df.total_income.fillna(total_income_median)


Строки c пропусками по столбцам `days_employed` и `total_income` могут быть полезны для исследования, а их точные значения не так важны. Заполним пропуски средним значением вместо нуля, что бы не создавать новые минимальные значения для столбцов. Некоторые значения в столбцах аномально велики что бы использовать метод mean() для определения среднего числа, поэтому будет лучше использовать метод median(). 

In [10]:
# Предобработка значений days_employed, приведение к модулю
df.days_employed = df.days_employed.apply(lambda x: abs(x))


Вероятно из-за технического сбоя системы сбора информации некоторые значения столбца `days_employed` были отрицательными.

In [11]:
# Вещественный тип данных total_income
df.dtypes


children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [12]:
# Целочисленный тип данных total_income
df.total_income = df.total_income.astype(int)
df.dtypes


children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income          int64
purpose              object
dtype: object

### Предобработка данных. Удаление дубликатов

In [13]:
# Поиск явных дупликатов
df.duplicated().sum()


54

In [14]:
# Устранение явных дупликатов
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()


0

In [15]:
# Создание функции unique_in_columns_searcher() для поиска аномальных значений в столбцах
def unique_in_columns_searcher(data):
    columns = list(data.columns)
    for value in columns:
        print(f'{value}:{data[value].sort_values().unique()}')
        print('')


In [16]:
# Применение функции unique_in_columns_searcher()
unique_in_columns_searcher(df)


children:[-1  0  1  2  3  4  5 20]

days_employed:[2.41416332e+01 2.42406948e+01 3.01953372e+01 ... 4.01675093e+05
 4.01715812e+05 4.01755400e+05]

dob_years:[ 0 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41
 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65
 66 67 68 69 70 71 72 73 74 75]

education:['ВЫСШЕЕ' 'Высшее' 'НАЧАЛЬНОЕ' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Начальное'
 'Неоконченное высшее' 'СРЕДНЕЕ' 'Среднее' 'УЧЕНАЯ СТЕПЕНЬ'
 'Ученая степень' 'высшее' 'начальное' 'неоконченное высшее' 'среднее'
 'ученая степень']

education_id:[0 1 2 3 4]

family_status:['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']

family_status_id:[0 1 2 3 4]

gender:['F' 'M' 'XNA']

income_type:['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']

debt:[0 1]

total_income:[  20667   21205   21367 ... 1726276 2200852 2265604]

purpose:['автомобили' 'автомобиль' 'высшее образо

In [17]:
# Результат поиска аномальных значений
print('С помощью самодельной функцией unique_in_columns_searcher() в таблице были обнаружены аномальные значения.')
print()
print('Обнаружены аномальные значения столбца children:') 
print(f'{df.children[df.children == -1].count()} строк с "-1" ребенком в семье и {df.children[df.children == 20].count()} строк с "20" детей в семье.')
print(f'Всего строк с аномальными значениями {df.children[df.children == -1].count() + df.children[df.children == 20].count()} из общего количства {df.shape[0]} строк в таблице.')
print()
print('Обнаружено аномальные значение "0" в столбце dob_years. Возраст заемщика не может быть равен 0.')
print(f'Всего {df.dob_years[df.dob_years==0].count()} строк со значением "0" из общего количства {df.shape[0]} строк в таблице.')


С помощью самодельной функцией unique_in_columns_searcher() в таблице были обнаружены аномальные значения.

Обнаружены аномальные значения столбца children:
47 строк с "-1" ребенком в семье и 76 строк с "20" детей в семье.
Всего строк с аномальными значениями 123 из общего количства 21471 строк в таблице.

Обнаружено аномальные значение "0" в столбце dob_years. Возраст заемщика не может быть равен 0.
Всего 101 строк со значением "0" из общего количства 21471 строк в таблице.


In [18]:
# Устранение неявных дупликатов
df = df[(df.children != -1) & (df.children != 20)]
df = df[df.dob_years != 0]
df.education = df.education.apply(lambda x: x.lower())
df.family_status = df.family_status.apply(lambda x: x.lower())
unique_in_columns_searcher(df)


children:[0 1 2 3 4 5]

days_employed:[2.41416332e+01 2.42406948e+01 3.01953372e+01 ... 4.01675093e+05
 4.01715812e+05 4.01755400e+05]

dob_years:[19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42
 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66
 67 68 69 70 71 72 73 74 75]

education:['высшее' 'начальное' 'неоконченное высшее' 'среднее' 'ученая степень']

education_id:[0 1 2 3 4]

family_status:['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем']

family_status_id:[0 1 2 3 4]

gender:['F' 'M' 'XNA']

income_type:['безработный' 'в декрете' 'госслужащий' 'компаньон' 'пенсионер'
 'предприниматель' 'сотрудник' 'студент']

debt:[0 1]

total_income:[  20667   21205   21367 ... 1726276 2200852 2265604]

purpose:['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомоб

Что бы найти неявные дубликаты я написал функцию `unique_in_columns_searcher()`. Она принимает таблицу с данными, создает список из названий столбцов таблицы, перебирает уникальные значения в столбцах и выводит результат на экран.

Столбик children (количество детей) имеет такие аномальные значения как "-1" и "20".
* "-1" ребёнок - 47 строк  и "20" детей - 76 из 21525 строк.
* "-1" ребенок в семье быть не может, а "20" - это неявное нарушение последовательности чисел в столбце. Возможно "20" - это "0", "2" или "6". 

Так как строк с этими значениями относительно общего массива меньше одного процента, лучше их исключить из исследования.

В столбце `dob_years` обнаружено аномальное значение "0". Возраст заемщика не может быть равен нулю. Возможно это значение указывает на то, что пользователь не указал свой возраст или данные были утеряны.
Так как строк с этим значением относительно общего массива меньше одного процента, лучше их исключить из исследования.

Столб `education` имел неявные дубликаты из-за ошибок ручного ввода значений, буквы разного размера. Что бы решить проблему, все значения были заменены на строчные.

Столб `family_status` имел одно значение с прописной буквы, в то время как все остальны начинались со строчной. Столб приведен к общему строчному виду.

In [19]:
# Данные о семейном статусе
df.pivot_table(index='family_status_id', values='family_status', aggfunc='first')


,family_status
family_status_id,
0,женат / замужем
1,гражданский брак
2,вдовец / вдова
3,в разводе
4,не женат / не замужем


In [20]:
# Данные об образовании
df.pivot_table(index='education_id', values='education', aggfunc='first')


,education
education_id,
0,высшее
1,среднее
2,неоконченное высшее
3,начальное
4,ученая степень


In [21]:
# Фильтрация таблицы df. Удаление столбцов education и family_status
df = df[['children', 'days_employed', 'dob_years', 'education_id', 'family_status_id', 'gender', 'income_type', 'debt', 'total_income', 'purpose']]


In [22]:
# Фильтрация таблицы df. Новый столбец таблицы df - total_income_category
# Значения: E, D, C, B, A

new_total_income_category_items = []
for value in df.total_income:
    if 0<=value<=30000:
        new_total_income_category_items.append('E')
    elif 30001<=value<=50000:
        new_total_income_category_items.append('D')
    elif 50001<=value<=200000:
        new_total_income_category_items.append('C')
    elif 200001<=value<=1000000:
        new_total_income_category_items.append('B')
    else:
        new_total_income_category_items.append('A')

df['total_income_category'] = new_total_income_category_items


In [23]:
# Фильтрация таблицы df. Новый столбец таблицы df - purpose_category
# значения: 'операции с автомобилем','операции с недвижимостью','проведение свадьбы','получение образования'
new_purpose_category_items = []
for value in df.purpose:
    if 'авто' in value:
        new_purpose_category_items.append('операции с автомобилем')
    elif 'жил' in value:
        new_purpose_category_items.append('операции с недвижимостью')
    elif 'недвиж' in value:
        new_purpose_category_items.append('операции с недвижимостью')
    elif 'свадь' in value:
        new_purpose_category_items.append('проведение свадьбы')
    elif 'образов' in value:
        new_purpose_category_items.append('получение образования')
    else:
        new_purpose_category_items.append('None')

df['purpose_category'] = new_purpose_category_items

# Проверка на наличие неучтенных значений
df.purpose_category.unique()


array(['операции с недвижимостью', 'операции с автомобилем',
       'получение образования', 'проведение свадьбы'], dtype=object)

**Итог**

Изменен тип данных с вещественного на целочисленный в столбце `total_income`.

Предобработка обнаружила две проблемы в данных:

- пропущенные значения,
- дубликаты — явные и неявные.

Устранены явные дупликаты. Пропущенные данные заполнены медианным значением.

Создана функция `unique_in_columns_searcher()` для поиска аномальных значений в столбцах.

Устранены неявные дупликаты в столбцах: 
* `children`, 
* `dob_years`, 
* `education`, 
* `family_status`.

Cтолбцы `education` и `family_status` для удобства заменены на `education_id` и `family_status_id`.

Добавлены новые столбы в таблицу df:
* `purpose_category` - категория целей получения кредита
* `total_income_category` - категория ежемесячных доходов

## Проверка гипотез

**Проверим следующие гипотезы:**
* Есть ли зависимость между количеством детей и возвратом кредита в срок?
* Есть ли зависимость между семейным положением и возвратом кредита в срок?
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
* Как разные цели кредита влияют на его возврат в срок?

In [24]:
# Фильтрация таблицы df под бизнес задачи
df = df[['children','family_status_id','debt','purpose_category','total_income_category']]
display(df.head())


,children,family_status_id,debt,purpose_category,total_income_category
0,1,0,0,операции с недвижимостью,B
1,1,0,0,операции с автомобилем,C
2,0,0,0,операции с недвижимостью,C
3,3,0,0,получение образования,B
4,0,1,0,проведение свадьбы,C


### Гипотеза: зависимость между количеством детей и возвратом кредита в срок.

Исследуем зависимость наличия задолженности от количества детей.

In [25]:
children = df.pivot_table(index='children', values='debt')
print(children)
print()
print('----------------Min-----------------')
print(children[children.debt == children.debt.min()])
print('----------------Max-----------------')
print(children[children.debt == children.debt.max()])


              debt
children          
0         0.075367
1         0.092009
2         0.095145
3         0.082317
4         0.097561
5         0.000000

----------------Min-----------------
          debt
children      
5          0.0
----------------Max-----------------
              debt
children          
4         0.097561


**Вывод**: *гипотеза отчасти подтвердилась, наличие детей влияет на возврат кредита в срок. Количество должников без детей меньше, чем должников с детьми. А вот разница между группами по количеству детей уже не такая значительная. В группе с 5 детьми мы видим значение равное нулю, скорее всего это означает клиенты у которых 5 детей не часто берут кредит и данные по этой группе не точные. Риск задолженности у всех групп клиентов с детьми.*

### Гипотеза: зависимость между семейным положением и возвратом кредита в срок.

Исследуем зависимость наличия задолженности от семейного положения.

	
family_status_id - family_status:
- 0	женат / замужем
- 1	гражданский брак
- 2	вдовец / вдова
- 3	в разводе
- 4	не женат / не замужем

In [26]:
family = df.pivot_table(index='family_status_id', values='debt')
print(family)
print()
print('----------------Min-----------------')
print(family[family.debt == family.debt.min()])
print('----------------Max-----------------')
print(family[family.debt == family.debt.max()])


                      debt
family_status_id          
0                 0.075544
1                 0.092848
2                 0.065539
3                 0.071247
4                 0.097842

----------------Min-----------------
                      debt
family_status_id          
2                 0.065539
----------------Max-----------------
                      debt
family_status_id          
4                 0.097842


**Вывод**: *гипотеза подтвердилась, семейный статус влияет на возврат кредита в срок. Клиенты реже остаются в задолженности если они относяткся к группам "вдова/вдовец", так же "разведен/разведена", "женат / замужем". К группам риска задолженности нужно отнести "не женат / не замужем" и "гражданский брак", так как обе группы близки к максимуму.*

### Гипотеза: зависимость между уровнем дохода и возвратом кредита в срок.

Исследуем зависимость наличия задолженности от уровня дохода.

`total_income_category` с категориями:
- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 и выше — 'A'.

In [27]:
total_income_category = df.pivot_table(index='total_income_category', values='debt')
print(total_income_category)
print()
print('----------------Min-----------------')
print(total_income_category[total_income_category.debt == total_income_category.debt.min()])
print('----------------Max-----------------')
print(total_income_category[total_income_category.debt == total_income_category.debt.max()])


                           debt
total_income_category          
A                      0.080000
B                      0.070784
C                      0.084830
D                      0.060519
E                      0.090909

----------------Min-----------------
                           debt
total_income_category          
D                      0.060519
----------------Max-----------------
                           debt
total_income_category          
E                      0.090909


**Вывод**: *существует некоторая зависимость от уровня дохода по группам, но определенной тендеции не обнаружено. Лучше всех справляются с кредитом клиенты с доходом 30001–50000. В группе риска клиенты с доходом 0–30000. Возможно это обусловлено тем, что зарплаты 0–30000 слишком низкие для региона проживания клиента что не позволяет гасить задолженность в срок, а зп 30001–50000 - средний уровень достатка.*

### Гипотеза: разные цели кредита влияют на его возврат в срок.

Исследуем зависимость наличия задолженности от целей кредита.

In [28]:
purpose_category = df.pivot_table(index='purpose_category', values='debt')
print(purpose_category)
print()
print('----------------Min-----------------')
print(purpose_category[purpose_category.debt == purpose_category.debt.min()])
print('----------------Max-----------------')
print(purpose_category[purpose_category.debt == purpose_category.debt.max()])


                              debt
purpose_category                  
операции с автомобилем    0.093192
операции с недвижимостью  0.072569
получение образования     0.092924
проведение свадьбы        0.078355

----------------Min-----------------
                              debt
purpose_category                  
операции с недвижимостью  0.072569
----------------Max-----------------
                            debt
purpose_category                
операции с автомобилем  0.093192


**Вывод**: *гипотеза подтвердилась, цель кредита влияет на его возврат в срок. Так клиенты с целью "операции с недвижимостью" реже остаются в долгах. В группе риска клиенты группы "операции с автомобилем", вероятно наличие автомобиля и его состояние напрямую влияют на возможности клиента гасить кредит.*

## Общий вывод:
На основе анализа четырех гипотез клиент с наибольшей вероятностью просрочки: 
- 4 детей, 
- не женат, 
- доход 0–30000, 
- цель кредита: операции с автомобилем
